In [49]:
import sys,importlib
sys.path.append('../')
import simulation
from numpy import *
import plotly.offline as py
import plotly.graph_objs as go

### sub-system

In [50]:
alpha = 1
delta = .25
gamma = .3
beta = -1
omega = 1
def dufffingOscillator(state,t):
    x,y = state[0],state[1]
    xdot = y
    ydot = -alpha*(x**3) - beta* x - delta*y + gamma*cos(omega*t)
    return asarray((xdot,ydot))

### coupling

In [57]:
a0,a1 = 1.5,1.2
w = 1.3

In [70]:
def coupling(stateA,stateB,t):
    x1,y1 = stateA
    x2,y2 = stateB
    #xdot = a1*w*x1*cos(w*t)+ y1*(a0+a1*sin(w*t)) -y2 -(x2-x1*(a0+a1*sin(w*t)))
    xdot = a1*w*x1*cos(w*t)# + (x2-x1*a1*sin(w*t))
    #ydot = a1*w*y1*cos(w*t) + alpha*x2**3 + beta*x2 + delta*y2 - gamma*cos(omega*t)+(a0+a1*sin(w*t))*(gamma*cos(omega*t)-alpha*x1**3-beta*x1-delta*y1) -(y2-y1*(a0+a1*sin(w*t)))
    ydot = (a1**3)*alpha*(x1**3)*(sin(w*t)**3) + a1*sin(w*t)*(beta*x1+delta*y1) + a1*w*y1*cos(w*t) + a1*sin(w*t)*(-alpha*x1**3-beta*x1-delta*y1+gamma*cos(omega*t)) - gamma*cos(omega*t) - (y2-y1*a1*sin(w*t))
    return asarray((xdot,ydot))

In [71]:
# def coupling(stateA,stateB,t):
#     x1,y1 = stateA
#     x2,y2 = stateB
#     xdot = a1*w*x1*cos(w*t)
#     ydot = a0*alpha*(x1**3) + a0*gamma*cos(w*t) + a1*alpha*(x1**3)*sin(w*t) + a1*gamma*sin(w*t)*cos(w*t) + a1*omega*y1*cos(w*t) + alpha*(x1**3)*((a0+a1*sin(w*t))**3) - gamma*cos(w*t)
#     return asarray((xdot,ydot))

### simulation

In [72]:
time = 50
delTime = .01
timeline = arange(0.0, time, delTime)

In [73]:
stateA0 = asarray([0,0])
stateB0 = asarray([0,0])
stateA,stateB = simulation.evolveAB(dufffingOscillator,dufffingOscillator,stateA0,stateB0,couplingB=coupling,time=time,delTime=delTime)

### synchronization - order parameter

In [74]:
py.iplot([go.Scatter(x=timeline,y=stateB[:,0]/stateA[:,0]),
         go.Scatter(x=timeline,y=stateB[:,1]/stateA[:,1])])

### eigenvalue evolution

In [47]:
# CHECK

def eigenvalue1(stateA,stateB,t):
    return -0.5*sqrt(-((a1**2)*sin(w*t)**2+1)*(12*(a1**4)*alpha*(stateA[:,0]**2)*(sin(w*t)**4) + 4*(a1**2)*beta*(sin(w*t)**2) - (a1**2)*(delta**2)*(sin(w*t)**2) + 12*alpha*(stateB[:,0]**2) + 4*beta - (delta**2))) - (delta+2)*((a1**2)*(sin(w*t)**2))/2

def eigenvalue2(stateA,stateB,t):
    return 0.5*sqrt(-((a1**2)*sin(w*t)**2+1)*(12*(a1**4)*alpha*(stateA[:,0]**2)*(sin(w*t)**4) + 4*(a1**2)*beta*(sin(w*t)**2) - (a1**2)*(delta**2)*(sin(w*t)**2) + 12*alpha*(stateB[:,0]**2) + 4*beta - (delta**2))) - (delta+2)*((a1**2)*(sin(w*t)**2))/2

In [48]:
py.iplot([go.Scatter(x=timeline,y=eigenvalue1(stateA,stateB,timeline)),
          go.Scatter(x=timeline,y=eigenvalue2(stateA,stateB,timeline))])